Tutorial Followed: https://medium.com/machinevision/implement-slam-from-scratch-b1fb599f40c8

In [30]:
import numpy as np

In [ ]:
class GraphSlam():
    def __init__(self, G):
        # Initialize information matrix and information vector
        n = len(G)  # Assuming number of nodes in G
        self.information_matrix = np.zeros((n, n)) # information matrix (omega) is of dims nxn, where n is the number of nodes
        self.information_vector = np.zeros((n, n)) # information vector (xi) is of dimensions nx1
    
    def graph_slam(self, G, start_pose):
        """
        Performs Graph SLAM on a graph G with starting pose.

        Args:
            G: A dictionary representing the graph, where keys are tuples representing
                (source, destination) and values are edge weights.
            start_pose: The initial pose of the robot.

        Returns:
            The optimized robot poses for all nodes in the graph.       """
        

        # Initial location constraint
        self.information_matrix[0, 0] = 1
        self.information_vector[0] = start_pose

        # Perform graph optimization
        optimized_poses = self.state_update(self.graph_optimization(self.information_matrix, self.information_vector, G))
        return optimized_poses

    def graph_optimization(self, information_matrix, information_vector, G):
        """
        Performs measurement and state update for graph optimization.

        Args:
            information_matrix: The information matrix.
            information_vector: The information vector.
            G: The graph dictionary.

        Returns:
            The updated information matrix and information vector.
        """
        information_matrix, information_vector = self.measurement_update(information_matrix, information_vector, G)
        optimized_poses = self.state_update(information_matrix, information_vector)
        return information_matrix, information_vector

    def measurement_update(self, information_matrix, information_vector, G):
        """
        Updates the information matrix and vector based on edge measurements.

        Args:
            information_matrix: The information matrix.
            information_vector: The information vector.
            G: The graph dictionary.

        Returns:
            The updated information matrix and information vector.
        """
        for edge, weight in G.items():
            source, destination = edge
            # Add information for both source and destination nodes
            information_matrix[source, source] += 1
            information_matrix[source, destination] -= 1
            information_vector[source] -= weight

            information_matrix[destination, destination] += 1
            information_matrix[destination, source] += 1
            information_vector[destination] += weight

        return information_matrix, information_vector

    def state_update(self, information_matrix, information_vector):
        """
        Calculates the state update (optimized poses) using the information matrix and vector.

        Args:
            information_matrix: The information matrix.
            information_vector: The information vector.

        Returns:
            The optimized robot poses for all nodes in the graph.
        """
        return np.linalg.inv(information_matrix) @ information_vector
